# Carga de dataset y funciones básicas

In [5]:
import tsplib95


import urllib.request
import gzip
import shutil

# Descargar el archivo comprimido
file = "swiss42.tsp"
urllib.request.urlretrieve(
    "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz",
    file + ".gz"
)

# Descomprimir usando Python
with gzip.open(file + ".gz", 'rb') as f_in:
    with open(file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Ahora sí puedes cargar el archivo
problem = tsplib95.load(file)
print("Nombre del problema:", problem.name)

Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())

import random
# Funciones básicas
# Se genera una solución aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
    solucion = [Nodos[0]]
    for n in Nodos[1:]:
        solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
    return solucion

# Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
    return problem.get_weight(a, b)

# Devuelve la distancia total de una trayectoria/solución
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion) - 1):
        distancia_total += distancia(solucion[i], solucion[i + 1], problem)
    distancia_total += distancia(solucion[-1], solucion[0], problem)
    return distancia_total

# Uso
sol_temporal = crear_solucion(Nodos)

ModuleNotFoundError: No module named 'tsplib95'

Búsqueda local

In [1]:
def busqueda_local(problem, N):
    Nodos = list(problem.get_nodes())
    current_solution = crear_solucion(Nodos)
    iteration = 0
    best_distance = distancia_total(current_solution, problem)
    best_solution = current_solution

    while iteration < N:
        index_a, index_b = random.sample(range(len(current_solution)), 2)
        temp_list = list(current_solution)

        #realizamos intercambio
        temp_list[index_a] = current_solution[index_b]
        temp_list[index_b] = current_solution[index_a]
        neighbor_distance = distancia_total(temp_list, problem)

        if neighbor_distance < best_distance:
          current_solution = list(temp_list)
          best_distance = neighbor_distance
        iteration += 1

    """ print("Best_solution:", best_solution)
    print("Best distance:", best_distance) """
    return best_solution, best_distance
        
""" 
Best_solution: [0, 31, 9, 12, 6, 37, 7, 27, 25, 34, 10, 38, 17, 36, 28, 39, 19, 22, 32, 16, 1, 26, 15, 24, 41, 40, 30, 14, 4, 21, 33, 18, 5, 2, 8, 20, 23, 11, 35, 29, 3, 13]
Best distance: 1691
 """
busqueda_local(problem,  5000)

NameError: name 'problem' is not defined

In [126]:
N_samples = (1000, 5000, 10000, 50000)
dict = { N:[] for N in list(N_samples)}

for N in N_samples:
  for test in range(200):
   dict[N].append(busqueda_local(problem, N)[1])

import pandas as pd

df = pd.DataFrame(dict)
stats = pd.DataFrame({
    'mean': df.mean(),
    'std': df.std(),
    'min': df.min(),
    'q75': df.quantile(0.75),
    'max': df.max()
})

display(stats)

,mean,std,min,q75,max
1000,2119.075,148.525785,1787,2215.75,2547
5000,1774.240,124.140350,1507,1852.25,2185
10000,1744.110,132.086152,1408,1829.50,2136
50000,1746.465,123.678781,1436,1830.25,2096


In [4]:
def genera_vecina(solucion):
    # Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) 
    # Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
    # print(solucion)
    mejor_solucion = []
    mejor_distancia = 100000

    for i in range(len(solucion) - 1):  # Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
        for j in range(i + 1, len(solucion)):

            # Se genera una nueva solución intercambiando los dos nodos i, j
            # (usamos el operador + que para listas en Python las concatena) 
            # ej.: [1,2] + [3] = [1,2,3]
            vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

            # Se evalúa la nueva solución ...
            distancia_vecina = distancia_total(vecina, problem)

            # ... para guardarla si mejora las anteriores
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina

    return mejor_solucion

# solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 4]
# print("Distancia Solución Inicial:", distancia_total(solucion, problem))

# Búsqueda Local:
# - Sobre el operador de vecindad 2-opt (función genera_vecina)
# - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
    mejor_solucion = []

    # Generar una solución inicial de referencia (aleatoria)
    solucion_referencia = crear_solucion(Nodos)
    mejor_distancia = distancia_total(solucion_referencia, problem)

    iteracion = 0  # Un contador para saber las iteraciones que hacemos
    while(1):
        iteracion += 1  # Incrementamos el contador
        # print('#', iteracion)

        # Obtenemos la mejor vecina ...
        vecina = genera_vecina(solucion_referencia)

        # ... y la evaluamos para ver si mejora respecto a lo encontrado hasta el momento
        distancia_vecina = distancia_total(vecina, problem)

        # Si no mejoramos hay que terminar. Hemos llegado a un mínimo local
        # (según nuestro operador de vecindad 2-opt)
        if distancia_vecina < mejor_distancia:
            # mejor_solucion = copy.deepcopy(vecina)  # Con copia profunda. Las copias en Python son por referencia
            mejor_solucion = vecina                 # Guarda la mejor solución encontrada
            mejor_distancia = distancia_vecina
        else:
            print("En la iteración", iteracion, ", la mejor solución encontrada es:", mejor_solucion)
            print("Distancia     :", mejor_distancia)
            return mejor_solucion

        solucion_referencia = vecina

# Llamada al algoritmo de búsqueda local
sol = busqueda_local(problem)


NameError: name 'problem' is not defined